In [1]:
from IPython.core.display import display, HTML
display(HTML('<style>.container {width:100%}</style>'))

In [2]:
import os,rich

In [3]:
import pandas as pd
import numpy as np

In [4]:
from pandas.api.types import CategoricalDtype

# 함수 정의

In [5]:
def f_comma(x):
    if isinstance(x, int):
        return '{:,}'.format(x)

In [6]:
def f_percent(x):
    if isinstance(x, int):
        return '{:,}%'.format(x)
    elif isinstance(x, float):
        return '{:.2f}%'.format(x)

## 1.데이터 프레임 만들기

### 1.1 개표 결과 데이터 프레임

In [7]:
url = 'https://github.com/algok2/Datasets/blob/main/TheElectionResults.xlsx?raw=true'
file_name = 'TheElectionResults.xlsx'
sheet_name = ['서울특별시','부산광역시','대구광역시','인천광역시','광주광역시','대전광역시','울산광역시','세종특별자치시','경기도','강원도','충청북도','충청남도','전라북도','전라남도','경상북도','경상남도','제주특별자치도']
col = ['시도','구시군명','선거인수','투표수','이재명', '윤석열', '심상정', '오준호', '허경영', '이백윤', '옥은호', '김경재', '조원진', '김재연','이경희', '김민찬','계','무효투표수','기권수','개표율']

In [8]:
df_gaepyo = pd.DataFrame(columns=pd.read_excel(file_name, sheet_name=0, header=3).columns)

for i in sheet_name:
#     print(i)
    temp = pd.read_excel(file_name, sheet_name=i, header=3)
    temp['시도'] = i
    df_gaepyo = df_gaepyo.append(temp)

In [9]:
col_map = {'Unnamed: 0':'구시군명', 'Unnamed: 1':'선거인수', 'Unnamed: 2':'투표수', 'Unnamed: 15':'계','Unnamed: 16':'무효투표수', 'Unnamed: 17':'기권수', 'Unnamed: 18':'개표율'}

df_gaepyo.rename(columns=col_map, inplace=True)

df_gaepyo.dropna(inplace=True)
df_gaepyo = df_gaepyo[df_gaepyo['구시군명']!='합계']
df_gaepyo.reset_index(drop=True, inplace=True)    
df_gaepyo.iloc[:,1:-1] = df_gaepyo.iloc[:,1:-1].astype('int32')

In [10]:
SUM_SUNGEOIN = df_gaepyo['선거인수'].sum() #총 선거인수
rich.print('{:<10} : {:,} 명'.format('선거인수',SUM_SUNGEOIN))

선거인수       : 44,197,692 명

In [11]:
df_a = pd.DataFrame(df_gaepyo.groupby(['시도'])['선거인수'].sum()).rename(columns = {'선거인수':'선거인수(명)'})
df_b = pd.DataFrame(df_a['선거인수(명)'].rank(method='min', ascending=False).astype('int32')).rename(columns = {'선거인수(명)':'RANK'})
df_c = pd.DataFrame(round(df_gaepyo.groupby(['시도'])['선거인수'].sum()/SUM_SUNGEOIN*100,2)).rename(columns = {'선거인수':'선거인수 비율(%)'})
df_a = pd.DataFrame(df_a['선거인수(명)'].apply(lambda x: f_comma(x)))
pd.concat([df_a, df_b, df_c], axis = 1)

,선거인수(명),RANK,선거인수 비율(%)
시도,,,
강원도,"1,333,621",12,3.02
경기도,"11,433,288",1,25.87
경상남도,"2,809,907",4,6.36
경상북도,"2,273,028",6,5.14
광주광역시,"1,209,206",14,2.74
대구광역시,"2,046,714",7,4.63
대전광역시,"1,233,177",13,2.79
부산광역시,"2,921,510",3,6.61
서울특별시,"8,346,647",2,18.88


### 윤석열 득표 수 - 이재명 득표 수

In [12]:
a = df_gaepyo['이재명'].sum()
b = df_gaepyo['윤석열'].sum()
c = df_gaepyo['무효투표수'].sum()
d = df_gaepyo['기권수'].sum()

rich.print('이재명 득표수 : {:10,} 표'.format(a))         #이재명 득표수
rich.print('윤석열 득표수 : {:10,} 표'.format(b))         #윤석열 득표수
rich.print('표         차 : {:10,} 표'.format(abs(a-b)))  #윤석열 득표 수 - 이재명 득표수
rich.print('[red]무 효 표   수 : {:10,} 표[/]'.format(abs(c))) #무효표수
rich.print('[red]기    권   수 : {:10,} 표[/]'.format(abs(d)))   #기권수

이재명 득표수 : 16,147,738 표

윤석열 득표수 : 16,394,815 표

표         차 :    247,077 표

무 효 표   수 :    307,542 표

기    권   수 : 10,129,839 표

### 득표율

In [13]:
SUM_YUHYO_TOPYOSU = df_gaepyo['계'].sum() #유효투표수 = 선거인수 - 기권수 - 무효투표수 
rich.print('유효표      : {0:10,} 표'.format(SUM_YUHYO_TOPYOSU))

유효표      : 33,760,311 표

In [14]:
a = df_gaepyo['이재명'].sum()/SUM_YUHYO_TOPYOSU*100
b = df_gaepyo['윤석열'].sum()/SUM_YUHYO_TOPYOSU*100

In [15]:
rich.print('이재명 득표율 : {0:10,} %'.format(a, end = ''))  #이재명 득표율
rich.print('윤석열 득표율 : {0:10,} %'.format(b, end = ''))  #윤석열 득표율
rich.print('득  표  율 차 : {0:10,} %p'.format(abs(a-b)))    #윤석열 득표율 - 이재명 득표율

이재명 득표율 : 47.83053686916569 %

윤석열 득표율 : 48.56239327890078 %

득  표  율 차 : 0.7318564097350873 %p

In [16]:
sido_order = CategoricalDtype(['전국','서울특별시','부산광역시','대구광역시','인천광역시','광주광역시','대전광역시','울산광역시','세종특별자치시','경기도','강원도','충청북도','충청남도','전라북도','전라남도','경상북도','경상남도','제주특별자치도'], ordered=True)

df_gaepyo['시도'] = df_gaepyo['시도'].astype(sido_order)

In [17]:
df_grp_1 = df_gaepyo[col[:-1]].groupby(['시도']).sum()

In [26]:
name = '윤석열'
df_a = pd.DataFrame(round(df_grp_1[name]/df_grp_1[name].sum()*100,2))
df_a = df_a[df_a.index != '전국']
df_b = pd.DataFrame(df_a[name].rank(method='min', ascending=False).astype('int32')).rename(columns = {name:'RANK'})
rich.print('전체 득표율(%)')
pd.concat([df_a, df_b], axis = 1).rename(columns = {name:name+'(%)'})

전체 득표율(%)

,윤석열(%),RANK
시도,,
서울특별시,19.86,2
부산광역시,7.75,4
대구광역시,7.32,6
인천광역시,5.36,7
광주광역시,0.76,16
대전광역시,2.83,11
울산광역시,2.42,12
세종특별자치시,0.62,17
경기도,24.19,1


In [38]:
a = 47.8 #이재명 득표율
b = 48.5 #윤석열 득표율

name = '윤석열'
df_a = pd.DataFrame(round(df_grp_1[name]/df_grp_1['계']*100,1), columns=[name]).dropna()
df_b = pd.DataFrame(df_a[name].dropna().rank(method='min', ascending=False).astype('int32')).rename(columns = {name:'RANK'})
df_c = pd.DataFrame((df_a >=a)*1).rename(columns = {name:'{}% 이상/이하'.format(b)})
rich.print('시도별 득표율(%)')
pd.concat([df_a, df_b, df_c], axis = 1).rename(columns = {name:name+'(%)'})

시도별 득표율(%)

,윤석열(%),RANK,48.5% 이상/이하
시도,,,
서울특별시,50.6,9,1
부산광역시,58.3,3,1
대구광역시,75.1,1,1
인천광역시,47.1,11,0
광주광역시,12.7,16,0
대전광역시,49.6,10,1
울산광역시,54.4,5,1
세종특별자치시,44.1,13,0
경기도,45.6,12,0


In [39]:
a = 47.8 #이재명 득표율
b = 48.5 #윤석열 득표율

name = '이재명'
df_a = pd.DataFrame(round(df_grp_1[name]/df_grp_1['계']*100,1), columns=[name]).dropna()
df_b = pd.DataFrame(df_a[name].dropna().rank(method='min', ascending=False).astype('int32')).rename(columns = {name:'RANK'})
df_c = pd.DataFrame((df_a >=a)*1).rename(columns = {name:'{}% 이상/이하'.format(a)})
rich.print('시도별 득표율(%)')
pd.concat([df_a, df_b, df_c], axis = 1).rename(columns = {name:name+'(%)'})

시도별 득표율(%)

,이재명(%),RANK,47.8% 이상/이하
시도,,,
서울특별시,45.7,9,0
부산광역시,38.2,14,0
대구광역시,21.6,17,0
인천광역시,48.9,7,1
광주광역시,84.8,2,1
대전광역시,46.4,8,0
울산광역시,40.8,13,0
세종특별자치시,51.9,5,1
경기도,50.9,6,1


## 1.2 주민등록 인구 데이터 프레임 만들기

In [153]:
df_jumin = pd.read_csv('행정구역_시군구_별_1세별_주민등록인구_2022.02.csv', encoding='cp949', header=1)

In [154]:
df_jumin

,행정구역(시군구)별,연령별,총인구수 (명),남자인구수 (명),여자인구수 (명)
0,전국,계,51625561,25738197,25887364
1,전국,0세,250466,128074,122392
2,전국,1세,271905,139370,132535
3,전국,2세,299103,153558,145545
4,전국,3세,328264,168411,159853
...,...,...,...,...,...
1831,제주특별자치도,96세,167,9,158
1832,제주특별자치도,97세,114,15,99
1833,제주특별자치도,98세,209,9,200
1834,제주특별자치도,99세,170,16,154


In [155]:
df_jumin_1 = df_jumin[df_jumin['연령별'] == '계'][['행정구역(시군구)별', '총인구수 (명)']]

In [156]:
df_jumin_1['총인구수 (명)'] = df_jumin_1['총인구수 (명)'].apply(lambda x: f_comma(x))
df_jumin_1

,행정구역(시군구)별,총인구수 (명)
0,전국,"51,625,561"
102,서울특별시,"9,508,451"
204,부산광역시,"3,347,396"
306,대구광역시,"2,381,095"
408,인천광역시,"2,950,978"
510,광주광역시,"1,440,032"
612,대전광역시,"1,450,862"
714,울산광역시,"1,119,446"
816,세종특별자치시,"376,399"
918,경기도,"13,574,369"


In [157]:
def f_age(x):
    x = x.split()[0]
    x = x.replace('세','')
    x = x.replace('계','')
    x = x.strip()
    if x == '':
        return 0
    else :
        return int(x)

In [158]:
df_jumin['age'] = df_jumin['연령별'].apply(lambda x: f_age(x))

In [159]:
df_jumin_vote = df_jumin[df_jumin['age'] >= 19]

In [149]:
df_jumin_vote = df_jumin_vote.groupby(['행정구역(시군구)별']).sum()['총인구수 (명)'].to_frame().rename(columns={'총인구수 (명)':'선거인수'})
# df_jumin_vote = df_jumin_vote.groupby(['행정구역(시군구)별']).sum()['총인구수 (명)'].apply(lambda x : f_comma(x)).to_frame().rename(columns={'총인구수 (명)':'선거인수'})

In [171]:
df_jumin_vote_1 = df_jumin_vote[df_jumin_vote['행정구역(시군구)별'] != '전국']
df_jumin_vote_1.groupby(['행정구역(시군구)별']).sum()['총인구수 (명)'].to_frame().sum() #43,692,910   44,197,692 명

총인구수 (명)    43692910
dtype: int64

In [172]:
44197692 - 43692910

504782

## 1.3 추계인구 데이터 프레임 만들기

In [107]:
df_chugae = pd.read_csv('성_및_연령별_추계인구_1세별__5세별__시도_20220314144023.csv', encoding='cp949', header=0)

In [115]:
df_chugae

,시나리오별(1),시도별(1),성별(1),연령별(1),연령별(2),2027,2032
0,중위 추계(기본 추계: 출산율-중위 / 기대수명-중위 / 국제이동-중위 / 국내이동...,전국,계,0 - 4세,0세,344572,352162
1,중위 추계(기본 추계: 출산율-중위 / 기대수명-중위 / 국제이동-중위 / 국내이동...,전국,계,0 - 4세,1세,332255,354312
2,중위 추계(기본 추계: 출산율-중위 / 기대수명-중위 / 국제이동-중위 / 국내이동...,전국,계,0 - 4세,2세,320345,355980
3,중위 추계(기본 추계: 출산율-중위 / 기대수명-중위 / 국제이동-중위 / 국내이동...,전국,계,0 - 4세,3세,308490,356185
4,중위 추계(기본 추계: 출산율-중위 / 기대수명-중위 / 국제이동-중위 / 국내이동...,전국,계,0 - 4세,4세,296953,351287
...,...,...,...,...,...,...,...
5395,중위 추계(기본 추계: 출산율-중위 / 기대수명-중위 / 국제이동-중위 / 국내이동...,제주특별자치도,여자,95 - 99세,95세,397,511
5396,중위 추계(기본 추계: 출산율-중위 / 기대수명-중위 / 국제이동-중위 / 국내이동...,제주특별자치도,여자,95 - 99세,96세,266,400
5397,중위 추계(기본 추계: 출산율-중위 / 기대수명-중위 / 국제이동-중위 / 국내이동...,제주특별자치도,여자,95 - 99세,97세,226,307
5398,중위 추계(기본 추계: 출산율-중위 / 기대수명-중위 / 국제이동-중위 / 국내이동...,제주특별자치도,여자,95 - 99세,98세,143,233


In [109]:
df_chugae_1 = df_chugae[df_chugae['성별(1)'] == '계'].groupby(['시도별(1)']).sum().reset_index()

In [110]:
df_chugae_1

,시도별(1),2027,2032
0,강원도,1514324,1521455
1,경기도,14109086,14375632
2,경상남도,3334331,3306304
3,경상북도,2605241,2577634
4,광주광역시,1438113,1405079
5,대구광역시,2321250,2258207
6,대전광역시,1451418,1429098
7,부산광역시,3172422,3072929
8,서울특별시,9271243,9073229
9,세종특별자치시,450221,504520


# 2. 데이터 프레임 합치기

In [111]:
df_merge = df_jumin_1.merge(df_chugae_1, left_on='행정구역(시군구)별', right_on='시도별(1)')[['시도별(1)','총인구수 (명)','2027','2032']].rename(columns={'총인구수 (명)':'2022'})
df_merge

,시도별(1),2022,2027,2032
0,전국,51625561,51925224,51844775
1,서울특별시,9508451,9271243,9073229
2,부산광역시,3347396,3172422,3072929
3,대구광역시,2381095,2321250,2258207
4,인천광역시,2950978,3007969,3041045
5,광주광역시,1440032,1438113,1405079
6,대전광역시,1450862,1451418,1429098
7,울산광역시,1119446,1110155,1089177
8,세종특별자치시,376399,450221,504520
9,경기도,13574369,14109086,14375632


In [112]:
df_merge['27년-22년'] = df_merge['2027'] - df_merge['2022']
df_merge['32년-22년'] = df_merge['2032'] - df_merge['2022']
df_merge

,시도별(1),2022,2027,2032,27년-22년,32년-22년
0,전국,51625561,51925224,51844775,299663,219214
1,서울특별시,9508451,9271243,9073229,-237208,-435222
2,부산광역시,3347396,3172422,3072929,-174974,-274467
3,대구광역시,2381095,2321250,2258207,-59845,-122888
4,인천광역시,2950978,3007969,3041045,56991,90067
5,광주광역시,1440032,1438113,1405079,-1919,-34953
6,대전광역시,1450862,1451418,1429098,556,-21764
7,울산광역시,1119446,1110155,1089177,-9291,-30269
8,세종특별자치시,376399,450221,504520,73822,128121
9,경기도,13574369,14109086,14375632,534717,801263


In [113]:
df_merge.set_index('시도별(1)', inplace=True)

In [114]:
df_merge_style = df_merge.style.format('{:,}')
df_merge_style

,2022,2027,2032,27년-22년,32년-22년
시도별(1),,,,,
전국,"51,625,561","51,925,224","51,844,775","299,663","219,214"
서울특별시,"9,508,451","9,271,243","9,073,229","-237,208","-435,222"
부산광역시,"3,347,396","3,172,422","3,072,929","-174,974","-274,467"
대구광역시,"2,381,095","2,321,250","2,258,207","-59,845","-122,888"
인천광역시,"2,950,978","3,007,969","3,041,045","56,991","90,067"
광주광역시,"1,440,032","1,438,113","1,405,079","-1,919","-34,953"
대전광역시,"1,450,862","1,451,418","1,429,098",556,"-21,764"
울산광역시,"1,119,446","1,110,155","1,089,177","-9,291","-30,269"
세종특별자치시,"376,399","450,221","504,520","73,822","128,121"
